# Machine Learning Foundation

## Section 2, Part a: Regression Intro: Transforming Target


## Learning objectives

By the end of this lesson, you will be able to:

*   Apply transformations to make target variable more normally distributed for regression
*   Apply inverse transformations to be able to use these in a regression context


In [1]:
pip install -U scikit-learn

     ---------------------------------------- 8.4/8.4 MB 902.4 kB/s eta 0:00:00
     -------------------------------------- 298.0/298.0 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Surpress warnings:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In the following cell we load the data and define some useful plotting functions.


In [3]:
np.random.seed(72018)

from sklearn.datasets import load_boston

def to_2d(array):
    return array.reshape(array.shape[0], -1)

def boston_dataframe(description=False):
    boston = load_boston()
    
    data = boston.data
    target = boston.target
    names = boston.feature_names
    
    target = to_2d(target)
    
    data_all = np.concatenate([data, target], axis=1)
    names_all = np.concatenate([names, np.array(['MEDV'])], axis=0)
    
    if description:
        
        return pd.DataFrame(data=data_all, columns=names_all), boston.DESCR
    
    else: 
        
        return pd.DataFrame(data=data_all, columns=names_all)
    
def plot_exponential_data():
    data = np.exp(np.random.normal(size=1000))
    plt.hist(data)
    plt.show()
    return data
    
def plot_square_normal_data():
    data = np.square(np.random.normal(loc=5, size=1000))
    plt.hist(data)
    plt.show()
    return data

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[:

### Loading in Boston Data


In [4]:
boston_data = boston_dataframe()

NameError: name 'boston_dataframe' is not defined

In [ ]:
boston_data.head(15)

### Determining Normality


Making our target variable normally distributed often will lead to better results

If our target is not normally distributed, we can apply a transformation to it and then fit our regression to predict the transformed values.

How can we tell if our target is normally distributed? There are two ways:

*   Visually
*   Using a statistical test


#### Visually


Plotting a histogram:


In [ ]:
boston_data.MEDV.hist();

Does not look normal due to that right tail. Let's try to verify statistically:


In [ ]:
from scipy.stats.mstats import normaltest # D'Agostino K^2 Test

Without getting into Bayesian vs. frequentist debates, for the purposes of this lesson, the following will suffice:

*   This is a statistical test that tests whether a distribution is normally distributed or not. It isn't perfect, but suffice it to say:
    *   This test outputs a "p-value". The *higher* this p-value is the *closer* the distribution is to normal.
    *   Frequentist statisticians would say that you accept that the distribution is normal (more specifically: fail to reject the null hypothesis that it is normal) if p > 0.05.


In [ ]:
normaltest(boston_data.MEDV.values)

p-value *extremely* low. Our y variable we've been dealing with this whole time was not normally distributed!


Linear Regression assumes a normally distributed residuals which can be aided by transforming y variable. Let's try some common transformations to try and get y to be normally distributed:

*   Log
*   Square root
*   Box cox


### Testing log


The log transform can transform data that is significantly skewed right to be more normally distributed:


In [ ]:
data = plot_exponential_data()

In [ ]:
plt.hist(np.log(data));

**Apply transform to Boston data:**


In [ ]:
log_medv = np.log(boston_data.MEDV)

In [ ]:
log_medv.hist();

In [ ]:
normaltest(log_medv)

Conclusion: closer, but still not normal.


### Exercise:

The square root transformation is another transformation that can transform non-normally distributed data into normally distributed data:


In [ ]:
data = plot_square_normal_data()

Slightly skewed right.


In [ ]:
plt.hist(np.sqrt(data));

Apply the square root transformation to the Boston data target and test whether the result is normally distributed.


In [ ]:
pass # your code here

In [ ]:
# Instructor Solution

sqrt_medv = np.sqrt(boston_data.MEDV)
plt.hist(sqrt_medv)

In [ ]:
normaltest(sqrt_medv)

### Box cox


The box cox transformation is a parametrized transformation that tries to get distributions "as close to a normal distribution as possible".

It is defined as:

$$ \text{boxcox}(y_i) = \frac{y_i^{\lambda} - 1}{\lambda} $$

You can think of as a generalization of the square root function: the square root function uses the exponent of 0.5, but box cox lets its exponent vary so it can find the best one.


In [ ]:
from scipy.stats import boxcox

In [ ]:
bc_result = boxcox(boston_data.MEDV)
boxcox_medv = bc_result[0]
lam = bc_result[1]

In [ ]:
lam

In [ ]:
boston_data['MEDV'].hist();

In [ ]:
plt.hist(boxcox_medv);

In [ ]:
normaltest(boxcox_medv)

Significantly more normally distributed (according to p value) than the other two distributions - above 0.05, even!

Now that we have a normally distributed y-variable, let's try a regression!


### Testing regression:


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (StandardScaler, 
                                   PolynomialFeatures)

In [ ]:
lr = LinearRegression()

**Reload clean version of `boston_data`:**


In [ ]:
boston_data = boston_dataframe()

Same steps as before.


**Create X and y**


In [ ]:
y_col = "MEDV"

X = boston_data.drop(y_col, axis=1)
y = boston_data[y_col]

**Create Polynomial Features**


In [ ]:
pf = PolynomialFeatures(degree=2, include_bias=False)
X_pf = pf.fit_transform(X)

**Train test split**


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_pf, y, test_size=0.3, 
                                                    random_state=72018)

**Fit `StandardScaler` on `X_train` as before**


In [ ]:
s = StandardScaler()
X_train_s = s.fit_transform(X_train)

**Discuss: what transformation do we need to apply next?**

Apply the appropriate transformation.


In [ ]:
pass # your code here

In [ ]:
# Instructor Solution
bc_result2 = boxcox(y_train)
y_train_bc = bc_result2[0]
lam2 = bc_result2[1]

As before, we'll now:

1.  Fit regression
2.  Transform testing data
3.  Predict on testing data


In [ ]:
y_train_bc.shape

In [ ]:
lr.fit(X_train_s, y_train_bc)
X_test_s = s.transform(X_test)
y_pred_bc = lr.predict(X_test_s)

### Discussion

*   Are we done?
*   What did we predict?
*   How would you interpret these predictions?


#### Inverse transform


Every transformation has an inverse transformation. The inverse transformation of $f(x) = \sqrt{x}$ is $f^{-1}(x) = x^2$, for example. Box cox has an inverse transformation as well: notice that we have to pass in the lambda value that we found from before:


In [ ]:
from scipy.special import inv_boxcox

In [ ]:
# code from above
bc_result = boxcox(boston_data.MEDV)
boxcox_medv = bc_result[0]
lam = bc_result[1]

In [ ]:
inv_boxcox(boxcox_medv, lam)[:10]

In [ ]:
boston_data['MEDV'].values[:10]

Exactly the same, as we would hope!


### Exercise:

1.  Apply the appropriate inverse transformation to `y_pred_bc`.
2.  Calculate the $R^2$ using the result of this inverse transformation and `y_test`.

**Hint:** Should be two lines of code.


In [ ]:
pass # your code here

In [ ]:
# Instructor Solution
y_pred_tran = inv_boxcox(y_pred_bc,lam2)
r2_score(y_pred_tran,y_test)

## LAB Exercise:

### Determine the R^2 of a LinearRegression without the box cox transformation. Is it higher or lower?


In [ ]:
### BEGIN SOLUTION
lr = LinearRegression()
lr.fit(X_train_s,y_train)
lr_pred = lr.predict(X_test_s)
r2_score(lr_pred,y_test)
### END SOLUTION

***

### Machine Learning Foundation (C) 2020 IBM Corporation
